In [1]:
from portfolio_optimization.data_processing import *
from portfolio_optimization.data_collection import *
from datetime import datetime, timezone, timedelta
from tokens.get_assets import *
import pandas as pd
import numpy as np

In [2]:
asset_list = ["btc"]
_df = get_historical_prices_for_assets(asset_list, time_range=timedelta(days=365 * 3), interested_columns=["ReferenceRate", "CapMrktEstUSD"])

# Filter out all columns containing `_` in their name
df = _df.loc[:, ~_df.columns.str.contains("_")]

# Get all the market caps
mcaps = _df.loc[:, _df.columns.str.contains("CapMrktEstUSD")]
mcaps.columns = mcaps.columns.str.replace("_CapMrktEstUSD", "")
mcaps.replace(np.nan, 0, inplace=True)
# Print comprehensive statistical summary
df

/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/data_collection/get_crypto_price_range.py:41: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/d2/1qh2h9ys19z3375prb_0cb8w0000gn/T/ipykernel_4049/951082817.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcaps.replace(np.nan, 0, inplace=True)


,btc
2020-10-04,10555.705434
2020-10-05,10665.618622
2020-10-06,10772.635600
2020-10-07,10596.373549
2020-10-08,10667.194394
...,...
2023-09-30,26911.251474
2023-10-01,26977.693313
2023-10-02,27950.145359
2023-10-03,27565.889295


In [3]:
from pycoingecko import CoinGeckoAPI

# Connect to CoinGecko API
cg = CoinGeckoAPI()

# Get historical data for BTC
data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days='max')

# Prepare the data for DataFrame
price_data = data['prices']
df_gecko = pd.DataFrame(price_data, columns=['time', 'price'])
df_gecko['time'] = pd.to_datetime(df_gecko['time'], unit='ms')  # convert the timestamp data to datetime
df_gecko.set_index('time', inplace=True)  # set the datetime as index
# Rename the column to BTC_gecko
df_gecko.rename(columns={'price': 'BTC_gecko'}, inplace=True)

df_gecko

,BTC_gecko
time,
2013-04-28 00:00:00,135.300000
2013-04-29 00:00:00,141.960000
2013-04-30 00:00:00,135.300000
2013-05-01 00:00:00,117.000000
2013-05-02 00:00:00,103.430000
...,...
2023-10-07 00:00:00,27958.196437
2023-10-08 00:00:00,27977.543491
2023-10-09 00:00:00,27948.103652


In [4]:
# Merge the two dataframes
df_merged = pd.merge(df, df_gecko, left_index=True, right_index=True)
df_merged

,btc,BTC_gecko
2020-10-04,10555.705434,10546.656255
2020-10-05,10665.618622,10670.700811
2020-10-06,10772.635600,10784.949403
2020-10-07,10596.373549,10605.957791
2020-10-08,10667.194394,10668.011011
...,...,...
2023-09-30,26911.251474,26917.199102
2023-10-01,26977.693313,26969.876144
2023-10-02,27950.145359,27967.510579
2023-10-03,27565.889295,27615.064885


In [5]:
# Add a column for the difference between the two prices
df_merged['diff'] = df_merged['btc'] - df_merged['BTC_gecko']
df_merged['diff_pct'] = df_merged['diff'] / df_merged['BTC_gecko']
df_merged

,btc,BTC_gecko,diff,diff_pct
2020-10-04,10555.705434,10546.656255,9.049179,8.580140e-04
2020-10-05,10665.618622,10670.700811,-5.082189,-4.762751e-04
2020-10-06,10772.635600,10784.949403,-12.313804,-1.141758e-03
2020-10-07,10596.373549,10605.957791,-9.584242,-9.036659e-04
2020-10-08,10667.194394,10668.011011,-0.816617,-7.654816e-05
...,...,...,...,...
2023-09-30,26911.251474,26917.199102,-5.947628,-2.209601e-04
2023-10-01,26977.693313,26969.876144,7.817169,2.898481e-04
2023-10-02,27950.145359,27967.510579,-17.365220,-6.209069e-04
2023-10-03,27565.889295,27615.064885,-49.175590,-1.780752e-03


In [6]:
# Statistics
df_merged.describe()

,btc,BTC_gecko,diff,diff_pct
count,1096.000000,1096.000000,1096.000000,1096.000000
mean,33222.519672,33247.833941,-25.314270,-0.000624
std,13425.554568,13447.903859,108.883472,0.002625
min,10555.705434,10546.656255,-819.013038,-0.017555
25%,22210.473401,22238.654475,-52.138303,-0.001596
50%,29785.959531,29810.245930,-9.757974,-0.000387
75%,42816.358362,42972.470309,9.192606,0.000349
max,67541.755508,67617.015545,967.592898,0.019350
